In [2]:
import io
import pandas as pd
import numpy as np
import sklearn
import requests
import tempfile
import os
import cv2
from datasets import load_dataset
from diffusers import StableDiffusionPipeline
import torch
from huggingface_hub import notebook_login
import shutil
import subprocess

# Functions used to retrieve image examples, format into a huggingface dataset, and push to a huggingface repo

In [108]:
def retrieve_class_images(class_prompt, class_data_dir, num_class_images):
    cmd = f"python /Users/megan.bultema/Documents/diffusers/examples/custom_diffusion/retrieve.py --class_prompt {class_prompt} --class_data_dir {class_data_dir} --num_class_images {num_class_images}"
    subprocess.run(cmd, shell=True)


In [133]:
def create_metadata_csv_and_push_to_hub(root: str, HF_user = 'megantron'):
    """
    Create a metadata.csv file and push it to the huggingface hub for a given image folder and text file. 
    This function creates a DataFrame with the image and text data from the text file and writes it to 
    metadata.csv. It also pushes the dataset to the Hugging Face hub.

    Parameters:
    -----------
    root : str
        Absolute path to the folder containing download of images using retrieve_class_images()

    """

    img_folder = root+'/images/'
    img_txtfile = root+'/images.txt'
    txt_file = root+'/caption.txt'
    # create a list of image file names from the folder
    img_files = os.listdir(img_folder)

    # create an empty list to store the image data
    img_data = [x for x in img_files]

    # read the text file and split the contents into a list
    with open(img_txtfile, 'r') as f:
        img_data = f.read().splitlines()

    #remove abs path to comply with hugging face upload
    img_file = [x.replace(f'{img_folder}','') for x in img_data]

    # read the text file and split the contents into a list
    with open(txt_file, 'r') as f:
        text_data = f.read().splitlines()

    
    # create a dictionary with the image and text data
    data_dict = {'file_name': img_file, 'text': text_data}

    # create a DataFrame from the dictionary
    df = pd.DataFrame(data_dict)

    df.to_csv(f'{img_folder}/metadata.csv', index = False)
    dataset = load_dataset("imagefolder", data_dir=img_folder)
    notebook_login()
    dataset.push_to_hub(f"{HF_user}/{root.split('/')[-1]}")


# Call above functions to prepare images for LoRA fine-tunning

In [109]:
retrieve_class_images("simpsons", "/Users/megan.bultema/Documents/image_diffusion/hackingtogether-megan/real_reg/simpsons2", 20)


downloading real regularization images: 100%|██████████| 20/20 [00:10<00:00,  1.91it/s]


In [134]:
create_metadata_csv_and_push_to_hub('/Users/megan.bultema/Documents/image_diffusion/hackingtogether-megan/real_reg/AnselAdams')

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /Users/megan.bultema/.cache/huggingface/datasets/imagefolder/default-708ea032dc8d4150/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

# Function to fine-tune model with LoRa using sripts available in the diffusers repo

In [3]:
def train_text_to_image_lora(MODEL_NAME, DATASET_NAME, OUTPUT_DIR):
    """
    Trains a text-to-image generation model using the specified parameters.

    Args:
        MODEL_NAME (str): The name or path of the pre-trained model to use.
        DATASET_NAME (str): The name or path of the dataset to use.
        OUTPUT_DIR (str): The output directory to save the trained model to.

    Returns:
        None
    """
    # Set environment variables
    os.environ["MODEL_NAME"] = MODEL_NAME
    os.environ["DATASET_NAME"] = DATASET_NAME
    os.environ["OUTPUT_DIR"] = OUTPUT_DIR

    # Launch training script
    !accelerate launch /Users/megan.bultema/Documents/diffusers/examples/text_to_image/train_text_to_image_lora.py \
        --pretrained_model_name_or_path=$MODEL_NAME \
        --dataset_name=$DATASET_NAME --caption_column="text" \
        --resolution=512 --random_flip \
        --train_batch_size=1 \
        --num_train_epochs=10 --checkpointing_steps=500 \
        --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
        --seed=42 \
        --output_dir=$OUTPUT_DIR


# Call above function for fine-tuning

In [4]:
train_text_to_image_lora("./converted_model_deliberate","megantron/simpsons_captions","deliberate_simpsons200")

/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/05/2023 13:44:07 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cpu

Mixed precision type: no

{'dynamic_thresholding_ratio', 'thresholding', 'variance_type', 'sample_max_value', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
{'cross_attention_norm', 'resnet_out_scale_factor', 'addition_embed_type', 'time_embedding_dim', 'resnet_skip_time_act', 'addition_embed_type_num_heads', 'encoder_hid_dim', 'class_embeddings_concat', 'mid_block_only_cross_attention', 'time_embedding_act_fn'} was not found in config. Values will be initialized to default values.
Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 829.24it/s]

Steps: 100%|█| 2000/2000 [44:12:32<00:00, 79.58s/it, lr=0.0001, step_loss=0.0729
Traceback (most recent call last):
  File "/Users/megan.bultema/opt/anaconda3/envs/diffpy2/bin/accelerate", line 10, in <module>
    sys.exit(main())
  File "/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/accelerate/commands/accelerate_cli.py", line 45, in main
    args.func(args)
  File "/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/accelerate/commands/launch.py", line 923, in launch_command
    simple_launcher(args)
  File "/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/accelerate/commands/launch.py", line 579, in simple_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/Users/megan.bultema/opt/anaconda3/envs/diffpy2/bin/python3.10', '/Users/megan.bultema/Documents/diffusers/examples/text_to_image/train_text_to_image_lora.py', '--pretr

# Function for generating and saving test images with the new fine-tuned weights

In [6]:
def generate_test_images(model_path: str, num_images, prompt, n_prompt) -> None:
    """
    Generate 5 test images using StableDiffusionPipeline and save them to a local directory named test_images.

    Args:
    - model_path (str): The path to the pre-trained model directory.
    num_images (int): Number of test images to generate

    Returns:
    - None
    """
    # check if test_images directory exists, if it does, remove it and create a new one
    if os.path.exists("test_images"):
        pass
    else:
        os.mkdir("test_images")

    # create StableDiffusionPipeline object and load attention processes
    pipe = StableDiffusionPipeline.from_pretrained("./converted_model_deliberate")
    pipe.unet.load_attn_procs(model_path)
    pipe.to("mps")

    # generate 5 test images and save them to test_images directory
    for i in range(num_images):

        image = pipe(prompt, negative_prompt=n_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
        save_header = model_path.split('/')[-1]
        image.save(f"test_images/{save_header}_test{i}.png")


# Below are calls to the above function to test various fine-tuned models

In [139]:
p = """(Cinematic photo: 1. 3), rat magician, directional look, 
octane render, ultra detailed, wide angle full body, 8k, ultra-detailed, (backlight:1. 2) intricate, style-empire"""
n = """nrealfixer, nfixer, nartfixer, illustration, drawing, 3d, b&w, 
(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, 
wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), 
disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation"""
generate_test_images("./deliberate_simpsons", 5, p, n)

/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [149]:
p = """ Mountains style Ansel Adams photography octane render, ultra detailed, wide angle full body, 8k, ultra-detailed, intricate, """
n = """nrealfixer, nfixer, nartfixer, illustration, drawing, 3d, b&w, 
(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, 
wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), 
disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation"""
generate_test_images("./deliberate_AnselAdams", 5, p, n)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
p = """ Bart Simpson as a military comander, ultra detailed, wide angle full body, 8k, ultra-detailed, intricate, """
n = """nrealfixer, nfixer, nartfixer, illustration, drawing, 3d, b&w, 
(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, 
wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), 
disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation"""
generate_test_images("./deliberate_simpsons200", 5, p, n)

/Users/megan.bultema/opt/anaconda3/envs/diffpy2/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/50 [00:00<?, ?it/s]